# Notebook com Testes para 2 amostras:
- [2 médias com amostras pareadas](#2mediasp)
- [2 variâncias](#2var)
- [2 médias de populações independentes - rodar teste variâncias primeiro](#2mediasi)
- [2 proporções](#2p)

In [1]:
#from scipy.stats import norm
import scipy.stats
from scipy.stats import t as t_student
import pandas as pd

In [2]:
def rej0(p, alfa):
    if (p<= alfa): r_teste= 'rejeitar'
    else: r_teste= 'não rejeitar'
    return r_teste

<a id='2mediasp '></a>
## Teste t-Student para 2 médias (amostras pareadas)
### 2 levantamentos de dados da mesma amostra (X, Y) com a interferência de um fator:
- comparar o peso antes e depois da dieta
- tempo de resposta de sistema antes e depois de modificações
- pressão arterial antes e depois da medicação
- ambas as populações devem ter distribuição normal

In [10]:
d= {'x': [16, 15, 11, 20, 19, 14, 13, 15, 14, 16],
    'y': [13, 13, 12, 16, 16, 11, 10, 15, 9, 16]}
alfa= 0.01
df= pd.DataFrame(d)
df['x-y']= df.x - df.y
df

,x,y,x-y
0,16,13,3
1,15,13,2
2,11,12,-1
3,20,16,4
4,19,16,3
5,14,11,3
6,13,10,3
7,15,15,0
8,14,9,5
9,16,16,0


In [11]:
mean_d= df['x-y'].mean()
std_d= df['x-y'].std()
n= df.shape[0]
T= df['x-y'].mean() / ( df['x-y'].std() / df.shape[0]**0.5 )
rv= t_student(n-1)
vlr_p= 2* rv.sf(abs(T))
t_alfa= rv.isf(alfa/2)
margem_err= abs(rv.ppf(alfa/2) * std_d / n**0.5)
intervalo_confianca= (round(mean_d-margem_err,4), round(mean_d+margem_err,4))
d= {'Média diferença':[mean_d], 'Tam Amostra':[n], 'std (x-y)':[std_d], 't_teste':[T], 
    'vlr_p':[vlr_p], 'Nivel de Significância':[alfa],'H0':['media_x = media_y'],'Teste H0':[rej0(vlr_p,alfa)], 
    'Interv. Conf':[intervalo_confianca]}
# aceitar H0 se intervalo_confianca contém zero
pd.DataFrame(d,index=['teste bicaudal'])

,Média diferença,Tam Amostra,std (x-y),t_teste,vlr_p,Nivel de Significância,H0,Teste H0,Interv. Conf
teste bicaudal,2.2,10,1.932184,3.600595,0.005743,0.01,media_x = media_y,rejeitar,"(0.2143, 4.1857)"


<a id='2var'></a>
## Teste F para 2 variâncias

In [3]:
from scipy.stats import f

In [4]:
def rejF(F, FL, FR):
    if (F < FL or F > FR): resp_F= 'rejeitar'
    else: resp_F= 'não rejeitar'
    return resp_F    

In [5]:
n1= 9                   # nr de elementos da amostra da população 1
var1= 2.2871**2             # variância da amostra da população 1
n2= 9                   # nr de elementos da amostra da população 2
var2= 2.7017**2             # variância da amostra da população 2
alfa= 0.05                # nivel de significância
#  H0: var1 = var2   e    Ha: var1 <> var2

In [6]:
F= var1 / var2
FL= 1 / f.isf(alfa/2, n2-1, n1-1)
FR= f.isf(alfa/2, n1-1, n2-1)
d= {'Nivel de Significância':[alfa], 'Tam Amostra1':[n1], 'Variância 1':[var1], 'Tam Amostra2':[n2], 'Variância 2':[var2], 
    'F':[F], 'FL':[FL], 'FR': [FR], 'H0':['variância_1 = variância_2'],'Teste H0':[rejF(F, FL, FR)]}
# rejeitar H0 se F < FL  ou   F > FR
pd.DataFrame(d,index=['teste bicaudal'])

,Nivel de Significância,Tam Amostra1,Variância 1,Tam Amostra2,Variância 2,F,FL,FR,H0,Teste H0
teste bicaudal,0.05,9,5.230826,9,7.299183,0.716632,0.225568,4.43326,variância_1 = variância_2,não rejeitar


<a id='2mediasi'></a>
## Após verificar se var1 = var2, podem ser comparadas as médias de populações independentes com as suposições:
- as 2 amostras são independentes
- as 2 amostras são extraídas aleatoriamente de populações distribuídas normalmente

In [10]:
def Rej(p, alfa):
    if (p<= alfa): r_teste= 'rejeitar'
    else: r_teste= 'não rejeitar'
    return r_teste 
d_cauda={'i':'Teste de cauda inferior', 's': 'Teste de cauda superior', 'b':'Teste bicaudal'} 
def Tp_testeM(cauda):
    if(cauda=='i'):    
        tp_teste=d_cauda.get(cauda)
        H0= 'media1 >= media2'
        Ha= 'media1 < media2 '
    elif(cauda=='s'):  
        tp_teste=d_cauda.get(cauda)
        H0= 'media1 <= media2'
        Ha= 'media1 > media2'
    elif(cauda== 'b'): 
        tp_teste=d_cauda.get(cauda)
        H0= 'media1 = media2'
        Ha= 'media1 <> media2'
    else: tp_teste='indefinido'
    l_teste= [H0,Ha,tp_teste]
    return l_teste

### [ir para: var1 <> var2 (Heterocedásticas)](#2mi_caso2)

## Comparar 2 médias de populações com var1 = var2 (Homocedásticas)

In [11]:
# nr de elementos e variância definidas na célula de teste das variâncias
m1= 18                            # média da amostra da população 1
m2= 21                            # média da amostra da população 2
cauda='b'
alfa= 0.05
## rodar a célula
var= ( (n1-1)*var1 + (n2-1)*var2 ) / ( n1-1 + n2-1 )
l_teste= Tp_testeM(cauda)   
d={'H0':[l_teste[0]], 'Ha':[l_teste[1]],'Tipo':[l_teste[2]], 'm1':[m1], 'n1':[n1], 'm2':[m2], 'n2':[n2], 'var':[var]}
pd.DataFrame(d, index=['Teste'])

,H0,Ha,Tipo,m1,n1,m2,n2,var
Teste,media1 = media2,media1 <> media2,Teste bicaudal,18,9,21,9,6.265005


In [12]:
t_teste= ( m1-m2 ) / ( (var/n1 + var/n2)**0.5 )
rv= t_student(n1+n2-2)
if(cauda=='b'): 
    p= 2* rv.sf(abs(t_teste))
    t_alfa= rv.isf(alfa/2)
    margem_err= abs(rv.ppf(alfa/2) * (var/n1 + var/n2)**0.5)
    intervalo_confianca= (round(m1-m2 -margem_err,4), round(m1-m2 +margem_err,4))
    d= {'var':[var], 'vlr_p':[p],
        'Nivel de Significância':[alfa],'t_teste':[t_teste], 't_alfa/2':[t_alfa],
        'H0':[l_teste[0]], 'Interv. Conf':[intervalo_confianca],'Teste H0':[Rej(p,alfa)]}
elif(cauda=='i'):  
    p= rv.cdf(t_teste)
    t_alfa= rv.ppf(alfa)                    # ppf inverse cdf
    d= {'var':[var], 'vlr_p':[p], 
        'Nivel de Significância':[alfa],'t_teste':[t_teste], 't_alfa':[z_alfa],
        'H0':[l_teste[0]], 'Teste H0':[Rej(p,alfa)]}
elif(cauda=='s'): 
    p= rv.sf(t_teste)
    t_alfa = rv.isf(alfa)
    d= {'var':[var], 'vlr_p':[p],
        'Nivel de Significância':[alfa],'t_teste':[t_teste], 't_alfa':[t_alfa],
        'H0':[l_teste[0]], 'Teste H0':[Rej(p,alfa)]}
else:
    p=0
    t_alfa= 0
    d= {'vlr_p':[p],'Nivel de Significância':[alfa],'t_teste':[t_teste], 't_alfa':[t_alfa], 
        'H0':[l_teste[0]], 'Teste H0':['Indefinido']}
pd.DataFrame(d, index=[d_cauda.get(cauda)]) 

,var,vlr_p,Nivel de Significância,t_teste,t_alfa/2,H0,Interv. Conf,Teste H0
Teste bicaudal,6.265005,0.021729,0.05,-2.542534,2.119905,media1 = media2,"(-5.5013, -0.4987)",rejeitar


### Teste bicaudal com intervalo de confiança:
- ic contém 0: m1=m2
- ic não contém0: m1 <> m2 com 1-alfa de confiança
- limites do ic negativos: m1 < m2
- limites do ic positivos: m1 > m2

<a id='2mi_caso2'></a>
## Comparar 2 médias de populações com var1 <> var2 (Heterocedásticas)

In [46]:
# nr de elementos e variância definidas na célula de teste das variâncias
m1= 227                            # média da amostra da população 1
m2= 449.3333                            # média da amostra da população 2
cauda='b'
alfa= 0.05
## rodar a célula
#var= ( (n1-1)*var1 + (n2-1)*var2 ) / ( n1-1 + n2-1 )
l_teste= Tp_testeM(cauda)   
d={'H0':[l_teste[0]], 'Ha':[l_teste[1]],'Tipo':[l_teste[2]], 'm1':[m1], 'n1':[n1], 'm2':[m2], 'n2':[n2]}
pd.DataFrame(d, index=['Teste'])

,H0,Ha,Tipo,m1,n1,m2,n2
Teste,media1 = media2,media1 <> media2,Teste bicaudal,227,9,449.3333,9


In [47]:
t_teste= ( m1-m2 ) / ( (var1/n1 + var2/n2)**0.5 )
rv= t_student(min(n1-1, n2-1))
if(cauda=='b'): 
    p= 2* rv.sf(abs(t_teste))
    t_alfa= rv.isf(alfa/2)
    margem_err= abs(rv.ppf(alfa/2) * (var1/n1 + var2/n2)**0.5)
    intervalo_confianca= (round(m1-m2 -margem_err,4), round(m1-m2 +margem_err,4))
    d= {'var':[var], 'vlr_p':[p],
        'Nivel de Significância':[alfa],'t_teste':[t_teste], 't_alfa/2':[t_alfa],
        'H0':[l_teste[0]], 'Interv. Conf':[intervalo_confianca],'Teste H0':[Rej(p,alfa)]}
elif(cauda=='i'):  
    p= rv.cdf(t_teste)
    t_alfa= rv.ppf(alfa)                    # ppf inverse cdf
    d= {'var':[var], 'vlr_p':[p], 
        'Nivel de Significância':[alfa],'t_teste':[t_teste], 't_alfa':[z_alfa],
        'H0':[l_teste[0]], 'Teste H0':[Rej(p,alfa)]}
elif(cauda=='s'): 
    p= rv.sf(t_teste)
    t_alfa = rv.isf(alfa)
    d= {'var':[var], 'vlr_p':[p],
        'Nivel de Significância':[alfa],'t_teste':[t_teste], 't_alfa':[t_alfa],
        'H0':[l_teste[0]], 'Teste H0':[Rej(p,alfa)]}
else:
    p=0
    t_alfa= 0
    d= {'vlr_p':[p],'Nivel de Significância':[alfa],'t_teste':[t_teste], 't_alfa':[t_alfa], 
        'H0':[l_teste[0]], 'Teste H0':['Indefinido']}
pd.DataFrame(d, index=[d_cauda.get(cauda)]) 

,var,vlr_p,Nivel de Significância,t_teste,t_alfa/2,H0,Interv. Conf,Teste H0
Teste bicaudal,0.425,0.000002,0.05,-12.447746,2.306004,media1 = media2,"(-263.5216, -181.145)",rejeitar


### Teste bicaudal com intervalo de confiança:
- ic contém 0: m1=m2
- ic não contém0: m1 <> m2 com 1-alfa de confiança
- limites do ic negativos: m1 < m2
- limites do ic positivos: m1 > m2

<a id='2p'></a>
## Teste Z para 2 proporções

In [13]:
d_cauda={'i':'Teste de cauda inferior', 's': 'Teste de cauda superior', 'b':'Teste bicaudal'}
def Tp_testep(cauda, p1, p2):
    if(cauda=='i'):    
        tp_teste=d_cauda.get(cauda)
        H0= 'p1 >= p2'
        Ha= 'p1 < p2'
    elif(cauda=='s'):  
        tp_teste=d_cauda.get(cauda)
        H0= 'p1 <= p2'
        Ha= 'p1 > p2'
    elif(cauda== 'b'): 
        tp_teste=d_cauda.get(cauda)
        H0= 'p1 = p2'
        Ha= 'p1 <> p2'
    else: tp_teste='indefinido'
    l_teste= [H0,Ha,tp_teste]
    return l_teste

In [16]:
n1= 140
p1= 34/n1
n2= 230
p2= 53/n2
cauda= 'b'
alfa= 0.1
## rodar a célula
p= (n1*p1 + n2*p2) / (n1 + n2)
l_teste= Tp_testep(cauda, p1, p2)   
d={'H0':[l_teste[0]], 'Ha':[l_teste[1]],'Tipo':[l_teste[2]], 'p1':[p1], 'n1':[n1], 'p2':[p2], 'n2':[n2], 'p':[p], 
   'n1p':[n1*p], 'n1(1-p)':n1*(1-p), 'n2p':[n2*p], 'n2(1-p)':n2*(1-p)}
# np e n(1-p) >=5                                   # distribuição amostral de p pode ser aproximada a uma distribuição normal
pd.DataFrame(d, index=['Teste'])

,H0,Ha,Tipo,p1,n1,p2,n2,p,n1p,n1(1-p),n2p,n2(1-p)
Teste,p1 = p2,p1 <> p2,Teste bicaudal,0.242857,140,0.230435,230,0.235135,32.918919,107.081081,54.081081,175.918919


In [17]:
z_teste= (p1 - p2) / ( (p*(1-p)/n1 + p*(1-p)/n2)**0.5 )
rv= scipy.stats.norm(0,1)
if(cauda=='b'): 
    vlr_p= 2* rv.sf(abs(z_teste))
    z_alfa= rv.isf(alfa/2)
    margem_err= z_alfa * ( ( p*(1-p)/n1 + p*(1-p)/n2 )**0.5 )
    intervalo_confianca= (round(p1-p2 -margem_err,4), round(p1-p2 +margem_err,4))
    d= {'p1 Amostra':[p1], 'Tam1 Amostra':[n1],'p2 Amostra':[p2], 'Tam2 Amostra':[n2], 'vlr_p':[vlr_p],
        'Nivel de Significância':[alfa],'z_teste':[abs(z_teste)], 'z_alfa/2':[z_alfa],
        'H0':[l_teste[0]], 'Interv. Conf.':[intervalo_confianca], 'Teste H0':[rej0(vlr_p,alfa)]}
elif(cauda=='i'):  
    vlr_p= rv.cdf(z_teste)
    z_alfa= rv.ppf(alfa)                    # ppf inverse cdf
    d= {'p Amostra':[p], 'vlr_p':[vlr_p], 
        'Nivel de Significância':[alfa],'z_teste':[z_teste], 'z_alfa':[z_alfa],
        'H0':[l_teste[0]], 'Teste H0':[rej0(vlr_p,alfa)]}
elif(cauda=='s'): 
    vlr_p= rv.sf(z_teste)
    z_alfa = rv.isf(alfa)
    d= {'p Amostra':[p], 'vlr_p':[vlr_p],
        'Nivel de Significância':[alfa],'z_teste':[z_teste], 'z_alfa':[z_alfa],
        'H0':[l_teste[0]], 'Teste H0':[rej0(vlr_p,alfa)]}
else:
    vlr_p=0
    z_alfa= 0
    d= {'vlr_p':[vlr_p],'Nivel de Significância':[alfa],'z_teste':[z_teste], 'z_alfa':[z_alfa], 
        'H0':[l_teste[0]], 'Teste H0':['Indefinido']}
pd.DataFrame(d, index=[d_cauda.get(cauda)])

,p1 Amostra,Tam1 Amostra,p2 Amostra,Tam2 Amostra,vlr_p,Nivel de Significância,z_teste,z_alfa/2,H0,Interv. Conf.,Teste H0
Teste bicaudal,0.242857,140,0.230435,230,0.784651,0.1,0.273263,1.644854,p1 = p2,"(-0.0624, 0.0872)",não rejeitar


### Teste bicaudal com intervalo de confiança:
- ic contém 0: p1=p2
- ic não contém0: p1 <> p2 com 1-alfa de confiança
- limites do ic negativos: p1 < p2
- limites do ic positivos: p1 > p2

In [18]:
# add dif para ver o ic